<h3>准备数据</h3>

<h4>读取数据</h4>

In [27]:
from time import localtime, strftime
import requests
import json
import pandas as pd
from pandas import DataFrame
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, Flatten, BatchNormalization
from tensorflow.keras.losses import Huber
from tensorflow.keras.regularizers import L1L2, L1, L2
from tensorflow.keras.callbacks import History, TensorBoard
from tensorflow.keras.backend import clear_session
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

<h4>设置tensorflow</h4>

In [6]:
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)

<h3>处理数据</h3>

<h4>读取数据</h4>

In [7]:
data = pd.read_csv("../data/pic/adjustment.tsv", delimiter="\t", skipinitialspace=True)
data.head(5)

,特征0,特征1,特征2,特征3,特征4,特征5,特征6,特征7,特征8,特征9,...,特征16,特征17,补偿0,补偿1,补偿2,补偿3,补偿4,补偿5,补偿6,补偿7
0,7.4,0.048,1.19,1.47,0.23,0.54,0.27,5.75,0.65,0.71,...,0.07,0.62,1.23,0.88,0.73,0.19,1.67,0.36,1.58,1.62
1,3.7,NaN,1.87,0.18,0.94,0.70,0.07,5.20,0.73,0.59,...,0.91,0.89,0.99,1.11,0.13,0.18,1.33,0.86,1.22,1.30
2,1.2,0.046,1.97,1.17,0.67,0.89,0.60,2.10,0.97,0.14,...,0.80,0.69,1.85,0.13,1.54,1.91,0.63,0.70,1.71,1.57
3,7.2,0.053,1.17,2.13,0.82,0.18,0.17,5.30,0.27,0.46,...,0.37,0.37,1.72,0.58,1.03,1.77,0.86,1.65,1.95,1.37
4,1.8,0.021,1.89,0.42,0.20,0.92,0.80,1.60,0.20,0.70,...,0.50,0.68,0.83,0.09,0.94,0.49,1.87,1.21,1.53,1.56


<h4>处理缺失值</h4>

In [8]:
data.dropna(axis=0, inplace=True)

In [9]:
x = data.iloc[:, 0:18]
y = data.iloc[:, 18:]
x.head(5)
y.head(5)

,补偿0,补偿1,补偿2,补偿3,补偿4,补偿5,补偿6,补偿7
0,1.23,0.88,0.73,0.19,1.67,0.36,1.58,1.62
2,1.85,0.13,1.54,1.91,0.63,0.70,1.71,1.57
3,1.72,0.58,1.03,1.77,0.86,1.65,1.95,1.37
4,0.83,0.09,0.94,0.49,1.87,1.21,1.53,1.56
5,0.62,0.33,1.13,1.67,1.12,1.23,1.55,1.22


<h4>划分验证集</h4>

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(3488, 18)
(873, 18)
(3488, 8)
(873, 8)


<h3>搭建模型并训练</h3>

<h4>搭建模型</h4>

In [14]:
model = Sequential([
    BatchNormalization(name="bn_0", input_shape=(x.shape[1], )),
    Dense(units=2048 * 4, activation=LeakyReLU(), kernel_regularizer=L2(0.2), activity_regularizer=L1(0.1), name="dense_1"),
    Dropout(0.3, name="dropout_1"),
    Dense(units=y.shape[1], activation=None, name="dense_8")
], name="adjustment")
model.compile(
    loss=Huber(),
    optimizer=Adam(learning_rate=1e-4),
    metrics=["mae"]
)

<h4>训练模型</h4>

In [15]:
history: History = model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_test, y_test),
    batch_size=32,
    epochs=240,
    verbose=1,
    workers=-1,
    use_multiprocessing=True,
    callbacks=[
        TensorBoard(log_dir="../logs/" + strftime("%Y%m%d-%H%M%S", localtime()))
    ]
)

Epoch 1/240
109/109 [==============================] - 1s 5ms/step - loss: 24.1370 - mae: 0.7782 - val_loss: 11.4370 - val_mae: 0.9299
Epoch 2/240
109/109 [==============================] - 0s 3ms/step - loss: 8.9508 - mae: 1.0282 - val_loss: 4.2711 - val_mae: 1.1599
Epoch 3/240
109/109 [==============================] - 0s 3ms/step - loss: 2.8702 - mae: 1.0604 - val_loss: 1.2228 - val_mae: 0.9934
Epoch 4/240
109/109 [==============================] - 0s 3ms/step - loss: 0.7421 - mae: 0.9797 - val_loss: 0.6182 - val_mae: 0.9771
Epoch 5/240
109/109 [==============================] - 0s 2ms/step - loss: 0.6114 - mae: 0.9696 - val_loss: 0.6085 - val_mae: 0.9667
Epoch 6/240
109/109 [==============================] - 0s 3ms/step - loss: 0.6024 - mae: 0.9593 - val_loss: 0.5991 - val_mae: 0.9564
Epoch 7/240
109/109 [==============================] - 0s 2ms/step - loss: 0.5940 - mae: 0.9491 - val_loss: 0.5912 - val_mae: 0.9464
Epoch 8/240
109/109 [==============================] - 0s 2ms/step 

Epoch 59/240
109/109 [==============================] - 0s 2ms/step - loss: 0.2996 - mae: 0.5886 - val_loss: 0.2994 - val_mae: 0.5889
Epoch 60/240
109/109 [==============================] - 0s 2ms/step - loss: 0.2964 - mae: 0.5843 - val_loss: 0.2920 - val_mae: 0.5846
Epoch 61/240
109/109 [==============================] - 0s 2ms/step - loss: 0.2930 - mae: 0.5800 - val_loss: 0.2936 - val_mae: 0.5803
Epoch 62/240
109/109 [==============================] - 0s 2ms/step - loss: 0.2884 - mae: 0.5759 - val_loss: 0.2857 - val_mae: 0.5763
Epoch 63/240
109/109 [==============================] - 0s 3ms/step - loss: 0.2840 - mae: 0.5719 - val_loss: 0.2830 - val_mae: 0.5722
Epoch 64/240
109/109 [==============================] - 0s 3ms/step - loss: 0.2822 - mae: 0.5679 - val_loss: 0.2806 - val_mae: 0.5684
Epoch 65/240
109/109 [==============================] - 0s 3ms/step - loss: 0.2787 - mae: 0.5641 - val_loss: 0.2772 - val_mae: 0.5647
Epoch 66/240
109/109 [==============================] - 0s 2ms

Epoch 120/240
109/109 [==============================] - 0s 2ms/step - loss: 0.1757 - mae: 0.4598 - val_loss: 0.1739 - val_mae: 0.4579
Epoch 121/240
109/109 [==============================] - 0s 2ms/step - loss: 0.1743 - mae: 0.4592 - val_loss: 0.1728 - val_mae: 0.4570
Epoch 122/240
109/109 [==============================] - 0s 2ms/step - loss: 0.1732 - mae: 0.4588 - val_loss: 0.1711 - val_mae: 0.4571
Epoch 123/240
109/109 [==============================] - 0s 2ms/step - loss: 0.1724 - mae: 0.4584 - val_loss: 0.1715 - val_mae: 0.4567
Epoch 124/240
109/109 [==============================] - 0s 2ms/step - loss: 0.1717 - mae: 0.4579 - val_loss: 0.1698 - val_mae: 0.4554
Epoch 125/240
109/109 [==============================] - 0s 2ms/step - loss: 0.1707 - mae: 0.4574 - val_loss: 0.1695 - val_mae: 0.4557
Epoch 126/240
109/109 [==============================] - 0s 3ms/step - loss: 0.1698 - mae: 0.4570 - val_loss: 0.1695 - val_mae: 0.4548
Epoch 127/240
109/109 [==============================] 

109/109 [==============================] - 0s 2ms/step - loss: 0.1477 - mae: 0.4511 - val_loss: 0.1461 - val_mae: 0.4489
Epoch 181/240
109/109 [==============================] - 0s 2ms/step - loss: 0.1478 - mae: 0.4511 - val_loss: 0.1463 - val_mae: 0.4489
Epoch 182/240
109/109 [==============================] - 0s 2ms/step - loss: 0.1480 - mae: 0.4511 - val_loss: 0.1461 - val_mae: 0.4489
Epoch 183/240
109/109 [==============================] - 0s 2ms/step - loss: 0.1480 - mae: 0.4511 - val_loss: 0.1461 - val_mae: 0.4489
Epoch 184/240
109/109 [==============================] - 0s 2ms/step - loss: 0.1480 - mae: 0.4511 - val_loss: 0.1463 - val_mae: 0.4489
Epoch 185/240
109/109 [==============================] - 0s 2ms/step - loss: 0.1482 - mae: 0.4511 - val_loss: 0.1466 - val_mae: 0.4489
Epoch 186/240
109/109 [==============================] - 0s 2ms/step - loss: 0.1483 - mae: 0.4511 - val_loss: 0.1465 - val_mae: 0.4490
Epoch 187/240
109/109 [==============================] - 0s 2ms/step 

In [ ]:
tensorboard --logdir ../logs

In [18]:
save_model(model, "/home/centos/adjustment/1")     # save_model(model, "../models/adjustment/1")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../logs/adjustments/1\assets


<h3>模型部署</h3>

<h4>部署服务</h4>

In [ ]:
!docker run -t \
    -p 8501:8501 \
    -v "/home/centos/adjustment:/models/adjustment" \
    -e MODEL_NAME=adjustment \
    tensorflow/serving:2.3.0

<h3>测试数据</h3>

<h4>读取数据并测试</h4>

In [45]:
data = x_test.values.tolist()
headers = {"content-type": "application/json"}

In [46]:
response = requests.post(
    url="http://81.70.8.71:8501/v1/models/adjustment:predict",
    data=json.dumps({"instances": data}),
    headers=headers
)
if response.status_code != 200:
    raise ValueError(response.text)

In [47]:
y_pred = np.array(json.loads(response.text)["predictions"])

In [48]:
print(mean_squared_error(y_test, y_pred))

0.2815481219531617
